## Topic_Modeling_Altmetric_Data_Processing

Prior to running this code, complete the these notebooks: 
* Topic_Modeling_Altmetric_Data_Loading


In [19]:
## General Dependencies
import re
import numpy as np
import pandas as pd
from pprint import pprint
import sys, os
import glob
from tika import parser # pip install tika
import inspect
import datetime
import pickle5 as pickle

## Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim import models
#from gensim.models.coherencemodel import CoherenceModel
from gensim.models import CoherenceModel
from gensim.models import LdaModel
from gensim.models.wrappers import LdaMallet
from gensim.models import ldaseqmodel


## Preprocessing
import spacy
import nltk as nltk
from nltk.stem import WordNetLemmatizer 
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

## Plotting
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import ast

## Other Libraries
from operator import itemgetter

## ScikitLearn
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

### Load data from previous notebooks

In [20]:
## Load data csv as a dataframe

final_df = pd.read_csv("output/loading/final_df.csv", index_col=0) 
final_df.head()


,metadata,content,status,title,file_name,Database,Document_Type,Reference_Detail,CORE,Authors,Title,Year,Publication_Date,Source_title,Page_start,File_Name,Metadata_Title
0,"{'Content-Type': 'application/pdf', 'Creation-...",\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r...,200,UCI MEDICAL CENTER AWARDED $1.3 MILLION FOR NA...,CO.105UCI MEDICAL CENTER AWARDED $1.3 MILLION ...,Nexis Uni,News Newswire,CO,NONCORE,NaN,UCI MEDICAL CENTER AWARDED $1.3 MILLION FOR NA...,1990,"December 4, 1990, Tuesday",PR Newswire,536 words,CO.105UCI MEDICAL CENTER AWARDED $1.3 MILLION ...,UCI MEDICAL CENTER AWARDED $1.3 MILLION FOR NA...
1,"{'Content-Type': 'application/pdf', 'Creation-...",\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r...,200,"KAISER PERMANENTE STUDY: IF YOU'RE OBESE, HOST...",CO.105KAISER PERMANENTE STUDY_ IF YOU'RE OBESE...,Nexis Uni,News Newswire,CO,NONCORE,NaN,"KAISER PERMANENTE STUDY: IF YOU'RE OBESE, HOST...",1994,"March 18, 1994, Friday",PR Newswire,380 words,CO.105KAISER PERMANENTE STUDY_ IF YOU'RE OBESE...,"KAISER PERMANENTE STUDY: IF YOU'RE OBESE, HOST..."
2,"{'Content-Type': 'application/pdf', 'Creation-...",\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r...,200,"YOUNG AMERICAN ADULTS BECOMING FATTER, NOT FIT...","CO.105YOUNG AMERICAN ADULTS BECOMING FATTER, N...",Nexis Uni,News Newswire,CO,NONCORE,NaN,"YOUNG AMERICAN ADULTS BECOMING FATTER, NOT FIT...",1995,"March 10, 1995, Friday",PR Newswire,749 words,"CO.105YOUNG AMERICAN ADULTS BECOMING FATTER, N...","YOUNG AMERICAN ADULTS BECOMING FATTER, NOT FIT..."
3,"{'Content-Type': 'application/pdf', 'Creation-...",\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r...,200,New Estimates On The Prevalence Of Hypertrophi...,CO.105New Estimates On The Prevalence Of Hyper...,Nexis Uni,News Newswire,CO,NONCORE,NaN,New Estimates On The Prevalence Of Hypertrophi...,1995,"August 30, 1995 Wednesday",Reuters Health Medical News,385 words,CO.105New Estimates On The Prevalence Of Hyper...,New Estimates On The Prevalence Of Hypertrophi...
4,"{'Content-Type': 'application/pdf', 'Creation-...",\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r...,200,KAISER PERMANENTE CORRECTS MISLEADING S.F. CHR...,CO.105KAISER PERMANENTE CORRECTS MISLEADING S....,Nexis Uni,News Newswire,CO,NONCORE,NaN,KAISER PERMANENTE CORRECTS MISLEADING S.F. CHR...,1995,"November 7, 1995, Tuesday",PR Newswire,1338 words,CO.105KAISER PERMANENTE CORRECTS MISLEADING S....,KAISER PERMANENTE CORRECTS MISLEADING S.F. CHR...


### Pre-process the text to lower case, remove special characters, etc.

In [21]:
## Pre-process the text to lower case, remove special characters, etc. 
## https://kavita-ganesan.com/extracting-keywords-from-text-tfidf/#.X7RHltBKiUn
## Test regex here: https://pythex.org/

def preprocess(text):
    
    ## Lowercase words
    text_lower = text.lower()
    
    ## Remove Emails from text
    ## if you need to match a \, you can precede them with a backslash to remove their special meaning: \\.
    ## \S matches any non-whitespace character; this is equivalent to the class [^ \t\n\r\f\v].
    ## \s Matches any whitespace character; this is equivalent to the class [ \t\n\r\f\v]
    ## Code below matches any character, then an @ sign, then more characters, end matching when a white space is found.
    text_email = re.sub('\\S*@\\S*\\s?', '', text_lower) 
    
    ## Remove URLS from text
    ## https://stackoverflow.com/questions/11331982/how-to-remove-any-url-within-a-string-in-python/40823105#40823105
    ## text_urls = re.sub(r'http\S+', '', text_email)
    ## https://www.geeksforgeeks.org/python-check-url-string/
    text_urls = re.sub(r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))",'', text_email)
    
    
    ## Remove tabs and new lines from text
    ## https://stackoverflow.com/questions/16355732/how-to-remove-tabs-and-newlines-with-a-regex
    ## \s Matches any whitespace character; this is equivalent to the class [ \t\n\r\f\v]
    text_spaces = re.sub(r'\s+',' ',text_urls)
        
    ## Remove \n from text
    text_space_character = text_spaces.replace('\n','')
    
    ## Remove \t from text
    text_tab_character = text_space_character.replace('\t','')
    
    ## Remove special characters and numbers
    ## \W matches any non-alphanumeric character; this is equivalent to the class [^a-zA-Z0-9_]
    ## \d matches any decimal digit; this is equivalent to the class [0-9]
    text_numbers = re.sub("(\\d|\\W)+"," ",text_tab_character)
    
    ## Remove tags
    ##text_tags = re.sub("","",text_numbers)

    ## Remove special characters and space, but leave in periods and numbers
    ## ^ means any character except. So [^5] will match any character except '5'
    ## [^a-zA-Z0-9_] matches any non-alphanumeric character.
    ## text_special = re.sub('[^A-Za-z0-9.]+|\s',' ',text_tab_character)
    
    ## Remove a sepcial list of terms
    ## The prune list is similar to "stop words" ... just easier to add/remove words on the fly
    ## https://stackoverflow.com/questions/15435726/remove-all-occurrences-of-words-in-a-string-from-a-python-list
    
    PRUNE_LIST = ['right reserved section',
                   'reserved section',
                   "length word byline", 
                   "byline", 
                   "word byline",
                   "journal code",
                   "load date", 
                   "english", 
                   "dr", 
                   "publication type magazine",
                   "type magazine",
                   "magazine",
                   "type newspaper",
                   "publication type newspaper",
                   'newspaper',
                   "group right reserved",
                   'section:',
                   'copyright',
                   'body',
                   'length:',
                   'keywords:',
                   'introduction',
                   'page',
                   'methodology',
                   'table',
                   'discussion',
                   'conclusions',
                   'references',
                   'classification',
                   'language',
                   'industry',
                   'geographic',
                   'load-date',
                   'end of document',
                   'mg dl',
                   'mg'
                   
                  ]

    remove = '|'.join(PRUNE_LIST)
    regex = re.compile(r'\b('+remove+r')\b', flags=re.IGNORECASE)
    text_special_remove = regex.sub("", text_numbers)

    return text_special_remove

## New column "preprocess" is formed from applying pre_process function to each item in the "content" column in dataframe
final_df['preprocess'] = final_df['content'].apply(lambda x:preprocess(x))

print(final_df['preprocess'][1])

#https://www.machinelearningplus.com/nlp/lemmatization-examples-python/

 kaiser permanente study if you re obese hostile or depressed try watching less tv kaiser permanente study if you re obese hostile or depressed try watching less tv pr newswire march friday eastern time  pr newswire association inc section lifestyle length words  a federally funded kaiser permanente study has found a link between watching television and obesity hostility and depression while heavy tv watchers were found to practice poor health habits associated with heart disease such as smoking the study could not determine if tv viewing actually causes heart attacks or heart disease the results of the study were presented today by kaiser permanente physician and lead investigator stephen sidney m d at the th annual conference on cardiovascular disease of epidemiology and prevention in tampa fla funded by the national institutes of health the cardia study examined television viewing and cardiovascular risk factors in young adults in four u s cities a bi racial population of young men 

### Tokenize the data using Gensim Utils Simple Preprocess

In [22]:
## Tokenize the data using Gensim Utils Simple Preprocess

def tokenize(text):
    token_list = gensim.utils.simple_preprocess(str(text), deacc=True)  # deacc=True removes punctuations
    return token_list

## New column "tokens" is formed from applying pre_process function to each item in the "content" column in dataframe
final_df['tokens'] = final_df['preprocess'].apply(lambda x:tokenize(x))


print(final_df['tokens'][1])

['kaiser', 'permanente', 'study', 'if', 'you', 're', 'obese', 'hostile', 'or', 'depressed', 'try', 'watching', 'less', 'tv', 'kaiser', 'permanente', 'study', 'if', 'you', 're', 'obese', 'hostile', 'or', 'depressed', 'try', 'watching', 'less', 'tv', 'pr', 'newswire', 'march', 'friday', 'eastern', 'time', 'pr', 'newswire', 'association', 'inc', 'section', 'lifestyle', 'length', 'words', 'federally', 'funded', 'kaiser', 'permanente', 'study', 'has', 'found', 'link', 'between', 'watching', 'television', 'and', 'obesity', 'hostility', 'and', 'depression', 'while', 'heavy', 'tv', 'watchers', 'were', 'found', 'to', 'practice', 'poor', 'health', 'habits', 'associated', 'with', 'heart', 'disease', 'such', 'as', 'smoking', 'the', 'study', 'could', 'not', 'determine', 'if', 'tv', 'viewing', 'actually', 'causes', 'heart', 'attacks', 'or', 'heart', 'disease', 'the', 'results', 'of', 'the', 'study', 'were', 'presented', 'today', 'by', 'kaiser', 'permanente', 'physician', 'and', 'lead', 'investigator

### Remove Stopwords using a custom stopword list

In [23]:
## Remove Stopwords using a custom stopword list


def remove_stopwords(text):
    
    ## Open stop words text file and save to stop_set variable
    with open("stop_words.txt", 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        f.close()

    ## The stopword list comes from the Terrier pacakge with 733 words and another 86 custom terms: 
    ## https://github.com/kavgan/stop-words/blob/master/terrier-stop.txt
    ## https://github.com/kavgan/stop-words/blob/master/minimal-stop.txt
    
    ## Other stopword list options can be reviewed here:
    ## https://medium.com/towards-artificial-intelligence/stop-the-stopwords-using-different-python-libraries-ffa6df941653

    ## Remove stop words from token_list
    ## https://stackoverflow.com/questions/29771168/how-to-remove-words-from-a-list-in-python
    token_nostop_list = [word for word in text if word not in stop_set]
        

    return token_nostop_list

## New column "no_stop" is formed from applying pre_process function to each item in the "content" column in dataframe
final_df['no_stop'] = final_df['tokens'].apply(lambda x:remove_stopwords(x))


print(final_df['no_stop'][1])

['kaiser', 'permanente', 'study', 'obese', 'hostile', 'depressed', 'try', 'watching', 'tv', 'kaiser', 'permanente', 'study', 'obese', 'hostile', 'depressed', 'try', 'watching', 'tv', 'pr', 'newswire', 'march', 'friday', 'eastern', 'time', 'pr', 'newswire', 'association', 'section', 'lifestyle', 'length', 'words', 'federally', 'funded', 'kaiser', 'permanente', 'study', 'found', 'link', 'watching', 'television', 'obesity', 'hostility', 'depression', 'heavy', 'tv', 'watchers', 'found', 'practice', 'poor', 'health', 'habits', 'associated', 'heart', 'disease', 'smoking', 'study', 'determine', 'tv', 'viewing', 'actually', 'causes', 'heart', 'attacks', 'heart', 'disease', 'results', 'study', 'presented', 'today', 'kaiser', 'permanente', 'physician', 'lead', 'investigator', 'stephen', 'sidney', 'th', 'annual', 'conference', 'cardiovascular', 'disease', 'epidemiology', 'prevention', 'tampa', 'fla', 'funded', 'national', 'institutes', 'health', 'cardia', 'study', 'examined', 'television', 'viewi

### Create Bigram and Trigram Tokens from non-stop word data

"Phrase modeling is another approach to learning combinations of tokens that together represent meaningful multi-word concepts. We can develop phrase models by looping over the the words in our corpus and looking for words that co-occur (i.e., appear one after another) together much more frequently than you would expect them to by random chance. The formula our phrase models will use to determine whether two tokens $A$ and $B$ constitute a phrase is:


                                                count(AB) - count_{min}
                                                - - - - - - - - - - - -   * N > threshold
                                                count(A) * count(B)

* count(A) is the number of times token $A$ appears in the corpus
* count(B) is the number of times token $B$ appears in the corpus
* count(AB) is the number of times the tokens $A\ B$ appear in the corpus in order
* N is the total size of the corpus vocabulary
* count_{min} is a user-defined parameter to ensure that accepted phrases occur a minimum number of times
* threshold is a user-defined parameter to control how strong of a relationship between two tokens the model requires before accepting them as a phrase

Once our phrase model has been trained on our corpus, we can apply it to new text. When our model encounters two tokens in new text that identifies as a phrase, it will merge the two into a single new token.

Phrase modeling is superficially similar to named entity detection in that you would expect named entities to become phrases in the model (so new york would become new_york). But you would also expect multi-word expressions that represent common concepts, but aren't specifically named entities (such as happy hour) to also become phrases in the model.

We turn to the indispensible gensim library to help us with phrase modeling — the Phrases class in particular."

Text from: <https://github.com/skipgram/modern-nlp-in-python/blob/master/executable/Modern_NLP_in_Python.ipynb>

In [24]:
# Convert the "no_stop" column in dataframe to a list to use in the build_bigrams_trigrams() function

def convert_nostop_to_list(final_df):
    
    nostop_list = []
    
    nostop_list = final_df['no_stop'].tolist()

    return nostop_list

nostop_list = convert_nostop_to_list(final_df)

In [25]:
## Create Bigram and Trigram Tokens from non-stop word data, and then compare to stopword

def build_bigrams_trigrams(text):
#     print("This is the text:", text)
#     print("---------------------------------------")

    
    ##Building Bigram & Trigram Models
    ##higher threshold fewer phrases.
    bigram = gensim.models.Phrases(text, min_count=5, threshold=100) 
    ## min_count: Ignore all words and bigrams with total collected count lower than this value.
    ## threshold: Represent a score threshold for forming the phrases (higher means fewer phrases).
    trigram = gensim.models.Phrases(bigram[text], threshold=100)
#     print(bigram)
#     print(trigram)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
#     print(bigram_mod)
#     print(trigram_mod)

    return bigram_mod, trigram_mod

bigram_mod, trigram_mod = build_bigrams_trigrams(nostop_list)

In [26]:
def make_bigrams(text, bigram_mod):
    bigram_token = []
    bigram_token.append(bigram_mod[text])
    
    return bigram_token

final_df['bigrams']  = final_df['no_stop'].apply(lambda x:make_bigrams(x, bigram_mod))

print(final_df['bigrams'][1])

[['kaiser_permanente', 'study', 'obese', 'hostile', 'depressed', 'try', 'watching_tv', 'kaiser_permanente', 'study', 'obese', 'hostile', 'depressed', 'try', 'watching_tv', 'pr_newswire', 'march', 'friday', 'eastern_time', 'pr_newswire', 'association', 'section', 'lifestyle', 'length_words', 'federally', 'funded', 'kaiser_permanente', 'study', 'found', 'link', 'watching_television', 'obesity', 'hostility', 'depression', 'heavy', 'tv', 'watchers', 'found', 'practice', 'poor', 'health', 'habits', 'associated', 'heart', 'disease', 'smoking', 'study', 'determine', 'tv_viewing', 'actually', 'causes', 'heart', 'attacks', 'heart', 'disease', 'results', 'study', 'presented', 'today', 'kaiser_permanente', 'physician', 'lead_investigator', 'stephen_sidney', 'th_annual', 'conference', 'cardiovascular', 'disease', 'epidemiology', 'prevention', 'tampa', 'fla', 'funded', 'national', 'institutes', 'health', 'cardia', 'study', 'examined', 'television_viewing', 'cardiovascular', 'risk', 'factors', 'youn

In [27]:
def make_trigrams(text, trigram_mod, bigram_mod ):
    trigram_token = []
    trigram_token.append(trigram_mod[bigram_mod[text]])
    return trigram_token

final_df['trigrams']  = final_df['no_stop'].apply(lambda x:make_trigrams(x, trigram_mod, bigram_mod))
print(final_df['trigrams'][1])

[['kaiser_permanente', 'study', 'obese', 'hostile', 'depressed', 'try', 'watching_tv', 'kaiser_permanente', 'study', 'obese', 'hostile', 'depressed', 'try', 'watching_tv', 'pr_newswire', 'march', 'friday', 'eastern_time', 'pr_newswire', 'association', 'section', 'lifestyle', 'length_words', 'federally', 'funded', 'kaiser_permanente', 'study', 'found', 'link', 'watching_television', 'obesity', 'hostility', 'depression', 'heavy', 'tv', 'watchers', 'found', 'practice', 'poor', 'health', 'habits', 'associated', 'heart', 'disease', 'smoking', 'study', 'determine', 'tv_viewing', 'actually', 'causes', 'heart', 'attacks', 'heart', 'disease', 'results', 'study', 'presented', 'today', 'kaiser_permanente', 'physician', 'lead_investigator', 'stephen_sidney', 'th_annual', 'conference', 'cardiovascular', 'disease', 'epidemiology', 'prevention', 'tampa', 'fla', 'funded', 'national', 'institutes', 'health', 'cardia', 'study', 'examined', 'television_viewing', 'cardiovascular', 'risk', 'factors', 'youn

In [28]:
## Create a dataframe with specific columns and send to csv for review
test_df = pd.DataFrame(final_df, columns = ['tokens','no_stop','bigrams','trigrams'])
test_df.to_csv('output/processing/nostop_ngrams.csv')

### Use Spacy for Parts of Speech Tagging 

Resource(s): 
<https://github.com/skipgram/modern-nlp-in-python/blob/master/executable/Modern_NLP_in_Python.ipynb>

In [29]:
def parts_of_speech(texts):
    
    parts_dict = {}
    nlp = spacy.load(r'C:\Users\keg827\AppData\Local\Continuum\anaconda3\Lib\site-packages\en_core_web_sm\en_core_web_sm-2.3.1')
    #nlp = spacy.load('C:\Users\keg827\AppData\Local\Continuum\anaconda3\Lib\site-packages\en_core_web_sm\en_core_web_sm-2.3.1', disable=['parser', 'ner'])
    nlp.max_length = 2700000 ## or any large value, as long as you don't run out of RAM
                    
    for doc in texts:
        ## Remove the commas between tokens, and apply spacy
        new_doc = nlp(" ".join(doc))
        ## Create a list for token and the token's part of speech 
        token_text = [token.orth_ for token in new_doc]
        token_pos = [token.pos_ for token in new_doc]
        ## Zip the two lists into a dictionary
        parts_dict= dict(zip(token_text, token_pos))
  
    return parts_dict

final_df['parts_of_speech']  = final_df['trigrams'].apply(lambda x:parts_of_speech(x))

## Review Spacy's parts of speech here: https://spacy.io/api/annotation
## E008 Text Length Exceeds Maximum Error: https://datascience.stackexchange.com/questions/38745/increasing-spacy-max-nlp-limit/55725

In [30]:
## Create a dataframe with specific columns and send to csv for review
test_df = pd.DataFrame(final_df, columns = ['tokens','no_stop','bigrams','trigrams', 'parts_of_speech'])
test_df.to_csv('output/processing/nostop_ngrams_partsofspeech.csv')

## Lemmatize the Data

"...lemmatization is a lot more powerful. It looks beyond word reduction and considers a language’s full vocabulary to apply a morphological analysis to words, aiming to remove inflectional endings only and to return the base or dictionary form of a word, which is known as the lemma."

Resource(s):
* <https://github.com/skipgram/modern-nlp-in-python/blob/master/executable/Modern_NLP_in_Python.ipynb>
* <https://www.geeksforgeeks.org/python-lemmatization-approaches-with-examples/>

In [31]:
## This step can take some time to complete

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV', 'PROPN']):
    lemmatize = []
    nlp = spacy.load(r'C:\Users\keg827\AppData\Local\Continuum\anaconda3\Lib\site-packages\en_core_web_sm\en_core_web_sm-2.3.1')
    #nlp = spacy.load('C:\Users\keg827\AppData\Local\Continuum\anaconda3\Lib\site-packages\en_core_web_sm\en_core_web_sm-2.3.1', disable=['parser', 'ner'])
    nlp.max_length = 2700000 #or any large value, as long as you don't run out of RAM
    
    for doc in texts:
        ## Remove the commas between tokens, and apply spacy
        new_doc = nlp(" ".join(doc))
        ## Add lemmatized tokens to the list if token's part of speech is in our allowed list 
        lemmatize.append([token.lemma_ for token in new_doc if token.pos_ in allowed_postags])
       
    return lemmatize

final_df['lemmatize']  = final_df['trigrams'].apply(lambda x:lemmatization(x, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV', 'PROPN']))

## Resources
## pip3 install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz
## https://stackoverflow.com/questions/54334304/spacy-cant-find-model-en-core-web-sm-on-windows-10-and-python-3-5-3-anacon

In [32]:
## Save work to CSV and inspect as needed
with open(r"output/processing/nostop_ngrams_partsofspeech_lemmatize.csv", 'w', encoding='utf-8') as file:
    test_df = pd.DataFrame(final_df, columns = ['tokens','no_stop','bigrams','trigrams', 'parts_of_speech', 'lemmatize'])
    test_df.to_csv(file, index=False, line_terminator='\n')
    file.close()   

In [33]:
# Convert the "lemmatize" column in dataframe to a list to provide to the get_gensim_corpus_dictionary() function

def convert_lemmatize_to_list(final_df):
     
    texts_out = final_df['lemmatize'].tolist()

    return texts_out

texts_out = convert_lemmatize_to_list(final_df)

## Inspect output as needed
# print(type(texts_out))
# print(texts_out)
# print(texts_out[1])
# print(type(texts_out[1]))


In [34]:
## Convert the output to a list of lists, not a LIST of lists of lists...which is what it was...
## This will be your final text data which will be used in the gensim topic modeling! 

texts_out_2 = [item for sublist in texts_out for item in sublist]

## Inspect the output as needed
print(type(texts_out_2))
# print(texts_out_2[1])
# print("----------------------------------------")
# print(texts_out_2[2])

<class 'list'>


In [35]:
## Save the list as a .pkl file

file_name = "output/processing/texts_out_2.pkl"

open_file = open(file_name, "wb")
pickle.dump(texts_out_2, open_file, protocol=4)
open_file.close()

## Resources
## https://www.kite.com/python/answers/how-to-save-and-read-a-list-in-python
## https://stackoverflow.com/questions/25843698/valueerror-unsupported-pickle-protocol-3-python2-pickle-can-not-load-the-file